In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("spam.csv",encoding="ISO-8859-1")

In [3]:
df=df.rename(columns={'v1':'target','v2':'email'})

In [4]:
df=df[['email','target']]

In [5]:
df

,email,target
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham
...,...,...
5567,This is the 2nd time we have tried 2 contact u...,spam
5568,Will Ì_ b going to esplanade fr home?,ham
5569,"Pity, * was in mood for that. So...any other s...",ham
5570,The guy did some bitching but I acted like i'd...,ham


# data cleaning

In [6]:
import re
def clean(str_):
  str_=str_.lower()
  str_=re.sub('[^A-Za-z0-9]+'," ",str_)
  return str_

In [7]:
df['cleaned_email']=df['email'].apply(clean)

<ipython-input-7-23148a7cb941>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cleaned_email']=df['email'].apply(clean)


In [8]:
sentences=df['cleaned_email'].tolist()

In [9]:
sent_tokens=[]
for sent in sentences:
  sent_tokens.append(sent.split(" "))

In [10]:
from gensim.models import Word2Vec

In [11]:
model=Word2Vec(sent_tokens, min_count=1, vector_size=100)

In [12]:
import numpy as np
sent_vec=[]
for sent in sentences:
  temp_vec=[]
  for word in sent.split(" "):
    temp_vec.append(model.wv[word])
  sent_vec.append(np.array(temp_vec).mean(axis=0))

In [13]:
df['feature']=sent_vec

In [14]:
df.head()

,email,target,cleaned_email,feature
0,"Go until jurong point, crazy.. Available only ...",ham,go until jurong point crazy available only in ...,"[-0.2978739, 0.39997745, -0.04561031, 0.056186..."
1,Ok lar... Joking wif u oni...,ham,ok lar joking wif u oni,"[-0.31330714, 0.42133394, -0.045494802, 0.0490..."
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam,free entry in 2 a wkly comp to win fa cup fina...,"[-0.31666037, 0.43955353, -0.051758394, 0.0520..."
3,U dun say so early hor... U c already then say...,ham,u dun say so early hor u c already then say,"[-0.43100563, 0.5778952, -0.065395944, 0.06718..."
4,"Nah I don't think he goes to usf, he lives aro...",ham,nah i don t think he goes to usf he lives arou...,"[-0.38427654, 0.49256012, -0.044604458, 0.0532..."


In [15]:
X=np.array(sent_vec)
Y=df['target']

In [16]:
X.shape

(5572, 100)

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2)

In [19]:
from sklearn.ensemble import RandomForestClassifier

In [20]:
clf=RandomForestClassifier()

In [21]:
clf.fit(X_train,y_train)

RandomForestClassifier()

In [22]:
y_pred=clf.predict(X_test)

In [23]:
from sklearn.metrics import accuracy_score

In [24]:
accuracy_score(y_test,y_pred)

0.9695067264573991

In [27]:
unknown_email="free free free hey you have won a mobile "

In [28]:
temp_vec
for word in unknown_email.split(" "):
  temp_vec.append(model.wv[word])
clf.predict([np.array(temp_vec).mean(axis=0)])

array(['spam'], dtype=object)